## 1.1 对poem.xlsx中的诗歌进行向前最大匹配分词，词表在wordlist.xlsx中已经给出

注意：有些字符可能不在词表中（out-of-vocabulary），可以保留这些字符，也可以直接去掉

In [2]:
# 读取文件，如有报错，可根据报错信息安装xlrd或者openpyxl
import pandas as pd
table = pd.read_excel('poem.xlsx')
word_list_df = pd.read_excel('wordlist.xlsx')

In [3]:
# 计算wordlist中词语的最大长度max_length
# TODO
max_length=0
for i in range(len(word_list_df)):
    max_length=max(max_length,len(word_list_df.word.iloc[i]))
print(max_length)

3


In [4]:
# 实现前向最大匹配分词算法
# string: 待分词的串
# word_list: 词表
# max_length: 词表中词语的最大长度
def cut(string, word_list, max_length):
    # TODO
    rlt=[]
    end_pos=min(max_length,len(string))
    start_pos=0
    while 1:
        if(start_pos==end_pos):
            start_pos+=1
            end_pos=min(start_pos+max_length,len(string))
        if(string[start_pos:end_pos] in word_list):
            rlt.append(string[start_pos:end_pos])
            start_pos=end_pos
            end_pos=min(start_pos+max_length,len(string))
        else:
            end_pos-=1
        if(start_pos==len(string)):
            break
    return rlt

In [5]:
word_list=set(word_list_df.word)

In [6]:
# 执行分词（可能需要等待几分钟）
table['words'] = table['content'].apply(lambda x: ' '.join(cut(x, word_list, max_length)))
table

,ID,Poem_id,line_number,content,words
0,1,4371,-100,##餞唐永昌( 一作餞唐郎中洛陽令),餞 唐 永昌 一作 餞 唐 郎中 洛陽 令
1,2,4371,-1,$$沈佺期,沈 期
2,3,4371,1,洛陽舊有( 一作出) 神明宰,洛陽 舊有 一作 出 神明 宰
3,4,4371,2,輦轂由來天地中,輦轂 由來 天地 中
4,5,4371,3,餘邑政成何足貴,餘 邑 政成 何足 貴
...,...,...,...,...,...
46272,46273,39205,-1,$$李舜弦,李 舜弦
46273,46274,39205,1,飲筵中散酒微醒,飲 筵 中 散 酒 微 醒
46274,46275,39205,2,濛濛雨草瑤階溼,濛濛 雨 草 瑤階 溼
46275,46276,39205,3,鐘曉愁吟獨倚屏,鐘 曉 愁吟 獨倚 屏


## 1.2 统计每个词的TF-IDF值

注意：本次作业实现最基础版本的TF-IDF计算即可，不必实现其他变种

In [7]:
# 按照空格分开，stack
split_words = table['words'].str.split(' ', expand=True).stack().rename('word').reset_index()
new_data = pd.merge(table['Poem_id'], split_words, left_index=True, right_on='level_0')
new_data

,Poem_id,level_0,level_1,word
0,4371,0,0,餞
1,4371,0,1,唐
2,4371,0,2,永昌
3,4371,0,3,一作
4,4371,0,4,餞
...,...,...,...,...
198498,39205,46275,4,屏
198499,39205,46276,0,盡日
198500,39205,46276,1,池邊
198501,39205,46276,2,釣


In [8]:
# 计算TF-IDF值
# TODO
import numpy as np
word_count=new_data.word.value_counts()
word_document_frequency={i:set() for i in new_data.dropna().word}
for i in range(new_data.shape[0]):
    if(not pd.isna(new_data.word[i])):
        word_document_frequency[new_data.word[i]].add(new_data.Poem_id[i])
poem_num=len(set(new_data.Poem_id))
tf_idf={i:(word_count[i]/len(new_data))*(np.log10(poem_num/len(word_document_frequency[i]))) for i in new_data.dropna().word}
tf_idf_df=pd.DataFrame.from_dict(tf_idf,orient='index',columns=['TF-IDF'])
tf_idf_df

,TF-IDF
餞,0.000241
唐,0.001558
永昌,0.000052
一作,0.008180
郎中,0.000595
...,...
髣佛,0.000020
竹錫,0.000020
青城,0.000020
物象,0.000020


In [9]:
# 